In [1]:
import pandas as pd

import api_requests as api
import helpers as helpers
from api_requests_mobilitat import *

In [2]:
# Test data
params = {"origin": "8503000", 
          "destination": "8507000", 
          "date": "2023-04-18", 
          "time": "13:07"}

In [3]:
# Use the function use_token() to get a token
api.use_token()

In [4]:
# Get a journey
journey = api.get_journey(origin=params['origin'], destination=params['destination'], date=params['date'], time=params['time'])

# Facts on the results
journey.keys()

dict_keys(['trips', 'paginationCursor'])

In [5]:
# Get all of the trips' durations
trip_durations = helpers.trip_durations(journey["trips"])

# Print them
for i, trip in enumerate(journey["trips"]):
    print(f"Duration of trip {i+1} :", trip["duration"], "so", trip_durations[i], "minutes.")

Duration of trip 1 : PT1H16M so 76 minutes.
Duration of trip 2 : PT56M so 56 minutes.
Duration of trip 3 : PT1H18M so 78 minutes.
Duration of trip 4 : PT1H28M so 88 minutes.
Duration of trip 5 : PT56M so 56 minutes.


In [6]:
place_params = {"longitude": 8.5441, 
                "latitude": 47.4115, 
                "radius": 1000, 
                "type": "StopPlace",
                "limit": 10, 
                "includeVehicleModes":True}

In [7]:
# Get nearby places
nearby_places = api.get_nearby_places(longitude=place_params['longitude'], latitude=place_params['latitude'], radius=place_params['radius'], type=place_params['type'], limit=place_params['limit'], includeVehicleModes=place_params['includeVehicleModes'])
get_train_stations = helpers.get_train_stations(nearby_places)
get_train_stations

[{'type': 'StopPlace',
  'id': '8503006',
  'name': 'Zürich Oerlikon',
  'canton': 'ZH',
  'centroid': {'type': 'Point', 'coordinates': [8.544112, 47.411527]},
  'distanceToSearchPosition': 3,
  'tariffBorder': False,
  'weighting': 12594,
  'vehicleModes': [{'id': 'BUS',
    'name': 'Bus',
    'corporateIdentityIcon': 'SBB_oev_b_t06',
    'vehicleSubModeName': 'Bus',
    'vehicleSubModeShortName': 'B'},
   {'id': 'TRAIN',
    'name': 'Train',
    'corporateIdentityIcon': 'SBB_oev_b_t02',
    'vehicleSubModeName': 'Night-urban train',
    'vehicleSubModeShortName': 'SN'},
   {'id': 'TRAIN',
    'name': 'Train',
    'corporateIdentityIcon': 'SBB_oev_b_t02',
    'vehicleSubModeName': 'RegioExpress',
    'vehicleSubModeShortName': 'RE'},
   {'id': 'TRAIN',
    'name': 'Train',
    'corporateIdentityIcon': 'SBB_oev_b_t02',
    'vehicleSubModeName': 'Urban train',
    'vehicleSubModeShortName': 'S'},
   {'id': 'TRAIN',
    'name': 'Train',
    'corporateIdentityIcon': 'SBB_oev_b_t02',
    '

In [8]:
# A way to check if a certain point has more than enough public transport stations nearby
# In that case, we can assume that the person lives in a big city and therefore doesn't need to take the car to get to the train station
# The person's coordinates [longitude, latitude]
coordinates = [6.561906, 46.518536]

# Check availability
helpers.is_applicable(coordinates, 5)

True

In [9]:
# Get nearby places
nearby_places = api.get_nearby_places(longitude=coordinates[0], latitude=coordinates[1], radius=1500, type="StopPlace", limit=50, includeVehicleModes=True)
train_stations = helpers.get_train_stations(nearby_places)
train_stations

[]

In [10]:
mobilitat_json = get_dataset()
columns = list(mobilitat_json[0].keys())
mobilitat_df = pd.json_normalize(mobilitat_json, meta = columns)
mobilitat_df.drop_duplicates(subset=["geopos.lon", "geopos.lat"], inplace=True)
mobilitat_df.head()

,didok,parkrail_anzahl,parkrail_preis_tag,parkrail_preis_monat,parkrail_preis_jahr,parkrail_pflichtig_zeit1,parkrail_pflichtig_zeit2,parkrail_pflichtig_zeit3,parkrail_bemerkung,railtaxitext,...,parkrail_automat,bpuic,bezeichnung_offiziell,abkuerzung,lod,tu_nummer,meteo,plz,geopos.lon,geopos.lat
0,10,NaN,NaN,NaN,NaN,None,None,None,Private Parkangebote: http://www.parkleitsyste...,Gratisnummer 0800 44 22 66,...,0.0,8500010,Basel SBB,BS,http://lod.opentransportdata.swiss/didok/8500010,11,https://meteo.sbb.ch/?didok=8500010,4051,7.589563,47.547412
32,16,NaN,NaN,NaN,NaN,None,None,None,None,Gratisnummer 0800 44 22 66,...,NaN,8500016,Basel St Johann,BSSJ,http://lod.opentransportdata.swiss/didok/8500016,11,https://meteo.sbb.ch/?didok=8500016,4056,7.572529,47.570306
33,20,31.0,5.0,50.0,500.0,00.00 - 24.00,00.00 - 24.00,00.00 - 24.00,None,Gratisnummer 0800 44 22 66,...,1.0,8500020,Muttenz,MU,http://lod.opentransportdata.swiss/didok/8500020,11,https://meteo.sbb.ch/?didok=8500020,4132,7.647894,47.533591
34,21,58.0,5.0,50.0,500.0,00.00 - 24.00,00.00 - 24.00,00.00 - 24.00,Diese Parkkasse ist in Pension. Ihre Nachfolge...,None,...,1.0,8500021,Pratteln,PR,http://lod.opentransportdata.swiss/didok/8500021,11,https://meteo.sbb.ch/?didok=8500021,4133,7.690817,47.522669
35,22,25.0,6.0,60.0,600.0,00.00 - 24.00,00.00 - 24.00,00.00 - 24.00,Weitere PP in Frenkendorf TOM: Mo-So 06.00 - 2...,None,...,1.0,8500022,Frenkendorf-Füllinsdorf,FRE,http://lod.opentransportdata.swiss/didok/8500022,11,https://meteo.sbb.ch/?didok=8500022,4402,7.719111,47.501470


In [11]:
# Finding stations from the [longitute, latitude] vector
position = [7.589563, 47.547412]

helpers.train_station_from_park_coords(position)

['8500010']

In [12]:
# Run this cell to create the mobilitat_df_with_closest_station dataframe and csv file

# Add number of train stations in the vicinity of the parking
"""
mobilitat_df["train_station_ids"] = mobilitat_df.apply(lambda x : train_station_from_park_coords([x["geopos.lon"], x["geopos.lat"]]), axis=1)
mobilitat_df_with_closest_stations = mobilitat_df

def append_strings(str_list):
    string_final = ""
    for string in str_list:
        string_final = string_final + "/" + string
    return string_final

mobilitat_df_with_closest_stations["train_station_ids"] = mobilitat_df_with_closest_stations.apply(lambda x : append_strings(x["train_station_ids"]), axis=1)
mobilitat_df_with_closest_stations.to_csv("mobilitat_df_with_closest_stations.csv")
"""

'\nmobilitat_df["train_station_ids"] = mobilitat_df.apply(lambda x : train_station_from_park_coords([x["geopos.lon"], x["geopos.lat"]]), axis=1)\nmobilitat_df_with_closest_stations = mobilitat_df\n\ndef append_strings(str_list):\n    string_final = ""\n    for string in str_list:\n        string_final = string_final + "/" + string\n    return string_final\n\nmobilitat_df_with_closest_stations["train_station_ids"] = mobilitat_df_with_closest_stations.apply(lambda x : append_strings(x["train_station_ids"]), axis=1)\nmobilitat_df_with_closest_stations.to_csv("mobilitat_df_with_closest_stations.csv")\n'

In [13]:
# Run this to read from the CSV file

mobilitat_df_with_closest_stations = pd.read_csv("mobilitat_df_with_closest_stations.csv", index_col = 0)
mobilitat_df_with_closest_stations["train_station_ids"] = mobilitat_df_with_closest_stations.apply(lambda x : helpers.divide_strings(x["train_station_ids"]), axis = 1)
mobilitat_df_with_closest_stations

,didok,parkrail_anzahl,parkrail_preis_tag,parkrail_preis_monat,parkrail_preis_jahr,parkrail_pflichtig_zeit1,parkrail_pflichtig_zeit2,parkrail_pflichtig_zeit3,parkrail_bemerkung,railtaxitext,...,bpuic,bezeichnung_offiziell,abkuerzung,lod,tu_nummer,meteo,plz,geopos.lon,geopos.lat,train_station_ids
0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private Parkangebote: http://www.parkleitsyste...,Gratisnummer 0800 44 22 66,...,8500010,Basel SBB,BS,http://lod.opentransportdata.swiss/didok/8500010,11,https://meteo.sbb.ch/?didok=8500010,4051,7.589563,47.547412,[8500010]
32,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gratisnummer 0800 44 22 66,...,8500016,Basel St Johann,BSSJ,http://lod.opentransportdata.swiss/didok/8500016,11,https://meteo.sbb.ch/?didok=8500016,4056,7.572529,47.570306,[8500016]
33,20,31.0,5.0,50.0,500.0,00.00 - 24.00,00.00 - 24.00,00.00 - 24.00,NaN,Gratisnummer 0800 44 22 66,...,8500020,Muttenz,MU,http://lod.opentransportdata.swiss/didok/8500020,11,https://meteo.sbb.ch/?didok=8500020,4132,7.647894,47.533591,[8500020]
34,21,58.0,5.0,50.0,500.0,00.00 - 24.00,00.00 - 24.00,00.00 - 24.00,Diese Parkkasse ist in Pension. Ihre Nachfolge...,NaN,...,8500021,Pratteln,PR,http://lod.opentransportdata.swiss/didok/8500021,11,https://meteo.sbb.ch/?didok=8500021,4133,7.690817,47.522669,[8500021]
35,22,25.0,6.0,60.0,600.0,00.00 - 24.00,00.00 - 24.00,00.00 - 24.00,Weitere PP in Frenkendorf TOM: Mo-So 06.00 - 2...,NaN,...,8500022,Frenkendorf-Füllinsdorf,FRE,http://lod.opentransportdata.swiss/didok/8500022,11,https://meteo.sbb.ch/?didok=8500022,4402,7.719111,47.501470,[8500022]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,16350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8516350,Buchrain,BURN,http://lod.opentransportdata.swiss/didok/8516350,11,https://meteo.sbb.ch/?didok=8516350,6030,8.356659,47.093650,[8516350]
801,16353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8516353,Baldegg Kloster,BLGK,http://lod.opentransportdata.swiss/didok/8516353,11,https://meteo.sbb.ch/?didok=8516353,6283,8.280598,47.180616,"[8516353, 8502025]"
802,16444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8516444,Kerzers Papiliorama,KZPP,http://lod.opentransportdata.swiss/didok/8516444,11,https://meteo.sbb.ch/?didok=8516444,3210,7.199281,46.989774,"[8516444, 8504401]"
803,17131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8517131,Pratteln Salina Raurica,PRSR,http://lod.opentransportdata.swiss/didok/8517131,11,https://meteo.sbb.ch/?didok=8517131,4133,7.709889,47.530588,"[8517131, 8500300]"


In [14]:
# From a position [longitude, latitude], get the closest train station ids and corresponding parking positions
position = [6.637078, 46.520714]
closest_train_stations_ids = helpers.get_closest_train_stations_from_departure(position)
closest_train_park_coords = helpers.get_closest_train_park_coords(closest_train_stations_ids)

In [17]:
corresponding_closest_parking_garades = helpers.get_closest_parking_garages_from_train_station(closest_train_stations_ids)

AttributeError: module 'helpers' has no attribute 'get_closest_parking_garages_from_train_station'